# VAE Example

From https://towardsdatascience.com/uncovering-anomalies-with-variational-autoencoders-vae-a-deep-dive-into-the-world-of-1b2bce47e2e9/


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DL4DS/sp2025_homeworks/blob/main/lecture_collateral/vae/03_vae.ipynb)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


## Simple VAE Trained on Random Input

In [2]:

# Define the VAE model
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        # Define the encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        # Define the latent representation
        self.fc_mu = nn.Linear(16, latent_dim)
        self.fc_logvar = nn.Linear(16, latent_dim)

        # Define the decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        z = self.reparameterize(mu, logvar)
        reconstructed = self.decoder(z)
        return reconstructed, mu, logvar

    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.randn_like(std)
        return mu + std*eps


In [3]:

# Train the VAE on the normal data
vae = VAE(input_dim=30, latent_dim=10)

# Generate random input data to test the model
data = torch.randn(100, 30)
optimizer = torch.optim.Adam(vae.parameters())

## Training Code

In [4]:
# Instantiate the model
model = VAE(input_dim=30, latent_dim=10)
optimizer = torch.optim.Adam(model.parameters())

# Generate random input data to test the model
# We run it through a sigmoid to ensure it is between 0 and 1
data = torch.sigmoid(torch.randn(100, 30))

# Define our reconstruction loss function
loss_fn = nn.BCELoss()


In [5]:

# Train the model
for epoch in range(100):

    # Compute the reconstruction loss
    reconstructed, mu, logvar = model(data)
    reconstruction_loss = loss_fn(reconstructed, data)

    # Compute the KL divergence loss
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Compute the total loss
    total_loss = reconstruction_loss + kl_loss

    # Backpropagate the gradients and update the model weights
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Print the loss values
    print(f"Epoch {epoch}: reconstruction_loss = {reconstruction_loss:.4f}, kl_loss = {kl_loss:.4f}, total_loss = {total_loss:.4f}")

Epoch 0: reconstruction_loss = 0.6982, kl_loss = 11.1982, total_loss = 11.8963
Epoch 1: reconstruction_loss = 0.6976, kl_loss = 10.3272, total_loss = 11.0248
Epoch 2: reconstruction_loss = 0.6970, kl_loss = 9.5823, total_loss = 10.2793
Epoch 3: reconstruction_loss = 0.6966, kl_loss = 8.9458, total_loss = 9.6423
Epoch 4: reconstruction_loss = 0.6967, kl_loss = 8.4035, total_loss = 9.1002
Epoch 5: reconstruction_loss = 0.6961, kl_loss = 7.9388, total_loss = 8.6349
Epoch 6: reconstruction_loss = 0.6960, kl_loss = 7.5405, total_loss = 8.2365
Epoch 7: reconstruction_loss = 0.6951, kl_loss = 7.1933, total_loss = 7.8884
Epoch 8: reconstruction_loss = 0.6958, kl_loss = 6.8829, total_loss = 7.5787
Epoch 9: reconstruction_loss = 0.6957, kl_loss = 6.5900, total_loss = 7.2856
Epoch 10: reconstruction_loss = 0.6955, kl_loss = 6.3087, total_loss = 7.0042
Epoch 11: reconstruction_loss = 0.6951, kl_loss = 6.0331, total_loss = 6.7282
Epoch 12: reconstruction_loss = 0.6954, kl_loss = 5.7469, total_loss 